#  1. Import Python libraries

In [1]:
pwd


'C:\\Users\\burak\\Desktop\\Burak\\itu\\3. Semester\\Project\\tahsin_fırıncı'

In [2]:
import os
import csv
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

import pandas as pd
import numpy as np

from PIL import Image
from random import shuffle

from skimage.feature import hog
from skimage.color import rgb2grey

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization
import numpy as np 

Using TensorFlow backend.


# 2. Create csv file

In [4]:
Ok_Images_Directory = r"C:\\Users\\burak\\Desktop\\Burak\\itu\\3. Semester\\Project\\tahsin_fırıncı\\OK_data_set\\"
NOk_Dents_Images_Directory = r"C:\\Users\\burak\\Desktop\\Burak\\itu\\3. Semester\\Project\\tahsin_fırıncı\\NOK_data_set\\"

#NOk_Pimples_Images_Directory = r"C:/Users/admin/Desktop/data_set/Pimples_2/"
#NOk_Scratch_Images_Directory = r"C:/Users/admin/Desktop/data_set/Scratch_3/"
#NOk_Dents_Images_Directory = r"C:/Users/admin/Desktop/data_set/Dents_1/"



In [5]:
os.listdir(Ok_Images_Directory)

['part5_ok_cam1_deg0_102.tiff',
 'part5_ok_cam1_deg0_103.tiff',
 'part5_ok_cam1_deg0_106.tiff',
 'part5_ok_cam1_deg0_107.tiff',
 'part5_ok_cam1_deg0_108.tiff',
 'part5_ok_cam1_deg0_109.tiff',
 'part5_ok_cam1_deg0_110.tiff',
 'part5_ok_cam1_deg0_134.tiff',
 'part5_ok_cam1_deg0_135.tiff',
 'part5_ok_cam1_deg0_136.tiff',
 'part5_ok_cam1_deg0_137.tiff',
 'part5_ok_cam1_deg0_138.tiff',
 'part5_ok_cam1_deg0_139.tiff',
 'part5_ok_cam1_deg0_140.tiff',
 'part5_ok_cam1_deg0_141.tiff',
 'part5_ok_cam1_deg0_142.tiff',
 'part5_ok_cam1_deg0_143.tiff',
 'part5_ok_cam1_deg0_144.tiff',
 'part5_ok_cam1_deg0_145.tiff',
 'part5_ok_cam1_deg0_146.tiff',
 'part5_ok_cam1_deg0_147.tiff',
 'part5_ok_cam1_deg0_148.tiff',
 'part5_ok_cam1_deg0_149.tiff',
 'part5_ok_cam1_deg0_150.tiff',
 'part5_ok_cam1_deg0_151.tiff',
 'part5_ok_cam1_deg0_152.tiff',
 'part5_ok_cam1_deg0_153.tiff',
 'part5_ok_cam1_deg0_154.tiff',
 'part5_ok_cam1_deg0_160.tiff',
 'part5_ok_cam1_deg0_164.tiff',
 'part5_ok_cam1_deg0_165.tiff',
 'part5_

In [6]:
dic={}
for filename in os.listdir(Ok_Images_Directory):
    dic[filename[0:-5]]=1
for filename in os.listdir(NOk_Dents_Images_Directory):
    dic[filename[0:-5]]=0
    
#for filename in os.listdir(Ok_Images_Directory):
 #   dic['Ok_'+filename]=1
#for filename in os.listdir(NOk_Pimples_Images_Directory):
 #   dic['NOk_'+"P_"+filename]=0
#for filename in os.listdir(NOk_Scratch_Images_Directory):
 #   dic['NOk_'+"S_"+filename]=0
#for filename in os.listdir(NOk_Dents_Images_Directory):
 #   dic['NOk_'+"D_"+filename]=0

In [7]:
dic["part5_ok_cam1_deg0_102"]

1

In [8]:
with open('mycsvfile.csv','wt') as f:
    w = csv.writer(f)
    w.writerows(dic.items())
    


# 3. Display image of each ok/nok metal sheet type

In [9]:
labels = pd.read_csv("mycsvfile.csv",names=["Image_File_Name", "class"], index_col=0)

In [10]:
labels.head()

,class
Image_File_Name,
part5_ok_cam1_deg0_102,1
part5_ok_cam1_deg0_103,1
part5_ok_cam1_deg0_106,1
part5_ok_cam1_deg0_107,1
part5_ok_cam1_deg0_108,1


In [11]:
def get_image(row_id, root="data_set/"):
    """
    Converts an image number into the file path where the image is located, 
    opens the image, and returns the image as a numpy array.
    """
    filename = "{}.tiff".format(row_id)
    file_path = os.path.join(root, filename)
    img = Image.open(file_path)
    grey_Ok_part = rgb2grey(np.array(img))
    
    return np.array(grey_Ok_part)

In [12]:
labels1=labels.reset_index()

In [13]:
img_list_arr=[]
for part_name in labels.index:
    img_list_arr.append([get_image(part_name),labels1[labels1.Image_File_Name==part_name]["class"].values])

In [14]:
img_list_arr = np.array([i[0] for i in img_list_arr]).reshape(-1,100, 100,1)

In [15]:
labels_arr = pd.get_dummies(labels['class'])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(img_list_arr, labels_arr, test_size=0.33, random_state=42)

In [17]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(100, 100, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(2, activation = 'softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 49, 49, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 23, 23, 64)        256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 96)        55392     
__________

In [20]:
model.fit(X_train, y_train.iloc[:].values, batch_size = 20, epochs = 10, verbose = 1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
134/134 [==============================] - 6s 43ms/step - loss: 0.7319 - acc: 0.5448
Epoch 2/10
134/134 [==============================] - 2s 17ms/step - loss: 0.5319 - acc: 0.7164
Epoch 3/10
134/134 [==============================] - 2s 17ms/step - loss: 0.5757 - acc: 0.7239
Epoch 4/10
134/134 [==============================] - 2s 17ms/step - loss: 0.6290 - acc: 0.6269
Epoch 5/10
134/134 [==============================] - 2s 18ms/step - loss: 0.4866 - acc: 0.7388
Epoch 6/10
134/134 [==============================] - 2s 17ms/step - loss: 0.4111 - acc: 0.7910
Epoch 7/10
134/134 [==============================] - 2s 18ms/step - loss: 0.4212 - acc: 0.7985
Epoch 8/10
134/134 [==============================] - 2s 17ms/step - loss: 0.4644 - acc: 0.7985
Epoch 9/10
134/134 [==============================] - 2s 17ms/step - loss: 0.4307 - acc: 0.8358
Epoch 10/10
134/134 [==============================] - 2s 17ms/step - loss: 0.4008 - acc

In [23]:
loss, acc = model.evaluate(X_test,y_test, verbose = 0)
print(acc * 100)

66.66666666666666
